In [1]:
import pandas as pd
from gurobipy import *

In [2]:
with open('test_case.txt') as text_file:
    data = text_file.read().splitlines()

data

['Test Sequence with length 20:',
 'CGUCUUCACUACAGCAUCGG',
 '',
 'Test Sequence with length 30:',
 'GACCUUACUGGGUACGAUUUACUGGAGGAC',
 '',
 'Test Sequence with length 50:',
 'GGCCAGACUGGUGGUGUGACUCCAGGCUAACCGGAUACGCGUGCCUCGGG',
 '',
 'Test Sequence with length 100:',
 'UAUGCAGGUCGCGUUUUUCCACUGCCUAGAUAGCUCUGAGGGUACACUUAGUUCAGCACAUAAGAGGGAUCAUACUAGGUCCGCGUCUUACCUCCUACGA',
 '',
 'Test Sequence with length 200:',
 'CCAUGUGGACGAGUUCACGUUGCGGUCUGCCGCGAAGAAUGAGUCGUCCAAUUUUUUUUUCCUGUAUGACGGAUAGAGUGUAGGGUGGGUGAUAGUGUAACUCCGUCCCGGCGUUGUCUAUGGCAGUCUAAUUGUAGUGAGCCAAAAUAUUUCAGAGAGUAACCAUUACAGGCUACCACACGACCCUGAUCCUAUUGUAU',
 '']

In [3]:
s = data[1]
len(s)

20

In [4]:
m = Model("Enhancements")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-12


In [5]:
m.setParam('OutputFlag',True)
m.setParam('TimeLimit', 5*60)

Set parameter TimeLimit to value 300


# Pre-Processing Data

In [6]:
ONE_N = list(range(1,len(s)+1))
ONE_N

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

## A_i, U_i, C_i, G_i

In [7]:
a = {}
u = {}
c = {}
g = {}

In [8]:
index = 1
for n in s:
    if (n == 'A'):
        a[index] = 1
        u[index] = 0
        c[index] = 0
        g[index] = 0
    if (n == 'U'):
        a[index] = 0
        u[index] = 1
        c[index] = 0
        g[index] = 0
    if (n == 'C'):
        a[index] = 0
        u[index] = 0
        c[index] = 1
        g[index] = 0
    if (n == 'G'):
        a[index] = 0
        u[index] = 0
        c[index] = 0
        g[index] = 1
    
    index += 1

In [9]:
print(a)
print(u)
print(c)
print(g)
print(s)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 0, 11: 1, 12: 0, 13: 1, 14: 0, 15: 0, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0}
{1: 0, 2: 0, 3: 1, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0}
{1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 1, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0}
{1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 1}
CGUCUUCACUACAGCAUCGG


## X_ij

In [10]:
ls_tup = []
for i in range(1,len(s)):
    for j in range(i+1,len(s)+1):
        ls_tup.append((i,j))

x = m.addVars(ls_tup,name='X_ij',vtype=GRB.BINARY)
x

{(1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (1, 19): <gurobi.Var *Awaiting Model Update*>,
 (1, 20): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>,
 (

## d_ij Minimum Distance Constraint

In [11]:
ls_dtup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_dtup.append((i,j))

d = {}

for i in ONE_N:
    for j in range(i+1,len(s)+1):
        w = min(j-i,len(s)-j+i)
        d[i,j] = w

d

{(1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (1, 5): 4,
 (1, 6): 5,
 (1, 7): 6,
 (1, 8): 7,
 (1, 9): 8,
 (1, 10): 9,
 (1, 11): 10,
 (1, 12): 9,
 (1, 13): 8,
 (1, 14): 7,
 (1, 15): 6,
 (1, 16): 5,
 (1, 17): 4,
 (1, 18): 3,
 (1, 19): 2,
 (1, 20): 1,
 (2, 3): 1,
 (2, 4): 2,
 (2, 5): 3,
 (2, 6): 4,
 (2, 7): 5,
 (2, 8): 6,
 (2, 9): 7,
 (2, 10): 8,
 (2, 11): 9,
 (2, 12): 10,
 (2, 13): 9,
 (2, 14): 8,
 (2, 15): 7,
 (2, 16): 6,
 (2, 17): 5,
 (2, 18): 4,
 (2, 19): 3,
 (2, 20): 2,
 (3, 4): 1,
 (3, 5): 2,
 (3, 6): 3,
 (3, 7): 4,
 (3, 8): 5,
 (3, 9): 6,
 (3, 10): 7,
 (3, 11): 8,
 (3, 12): 9,
 (3, 13): 10,
 (3, 14): 9,
 (3, 15): 8,
 (3, 16): 7,
 (3, 17): 6,
 (3, 18): 5,
 (3, 19): 4,
 (3, 20): 3,
 (4, 5): 1,
 (4, 6): 2,
 (4, 7): 3,
 (4, 8): 4,
 (4, 9): 5,
 (4, 10): 6,
 (4, 11): 7,
 (4, 12): 8,
 (4, 13): 9,
 (4, 14): 10,
 (4, 15): 9,
 (4, 16): 8,
 (4, 17): 7,
 (4, 18): 6,
 (4, 19): 5,
 (4, 20): 4,
 (5, 6): 1,
 (5, 7): 2,
 (5, 8): 3,
 (5, 9): 4,
 (5, 10): 5,
 (5, 11): 6,
 (5, 12): 7,
 (5, 13): 8,
 (5, 14): 9,

In [12]:
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if d[i,j] < 3:
            x[i,j] = 0

x

{(1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 7): <gurobi.Var *Awaiting Model Update*>,
 (1, 8): <gurobi.Var *Awaiting Model Update*>,
 (1, 9): <gurobi.Var *Awaiting Model Update*>,
 (1, 10): <gurobi.Var *Awaiting Model Update*>,
 (1, 11): <gurobi.Var *Awaiting Model Update*>,
 (1, 12): <gurobi.Var *Awaiting Model Update*>,
 (1, 13): <gurobi.Var *Awaiting Model Update*>,
 (1, 14): <gurobi.Var *Awaiting Model Update*>,
 (1, 15): <gurobi.Var *Awaiting Model Update*>,
 (1, 16): <gurobi.Var *Awaiting Model Update*>,
 (1, 17): <gurobi.Var *Awaiting Model Update*>,
 (1, 18): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 7): <gurobi.Var *Awaiting Model Update*>,
 (2, 8): <gurobi.Var *Awaiting Model Update*>,
 (2, 9): <gurobi.Var *Awaiting Model Update*>,
 (2, 10): <gurobi.Var *Awaiting Model Update*>,
 (2

## f_ij

In [13]:
ls_ftup = []
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        ls_ftup.append((i,j))

f = {}
f

{}

In [14]:
temp = ''
temp = 'x' + s
temp
for i in ONE_N:
    for j in range(i+1,len(s)+1):
        if (((temp[i] == 'C') & (temp[j] == 'G')) or ((temp[i] == 'G') & (temp[j] == 'C'))):
            f[i,j] = 3
        elif (((temp[i] == 'A') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'A'))):
            f[i,j] = 2
        elif (((temp[i] == 'G') & (temp[j] == 'U')) or ((temp[i] == 'U') & (temp[j] == 'G'))):
            f[i,j] = 0.1
        elif (((temp[i] == 'A') & (temp[j] == 'C')) or ((temp[i] == 'C') & (temp[j] == 'A'))):
            f[i,j] = 0.05
        else:
            f[i,j] = 0
            
f

{(1, 2): 3,
 (1, 3): 0,
 (1, 4): 0,
 (1, 5): 0,
 (1, 6): 0,
 (1, 7): 0,
 (1, 8): 0.05,
 (1, 9): 0,
 (1, 10): 0,
 (1, 11): 0.05,
 (1, 12): 0,
 (1, 13): 0.05,
 (1, 14): 3,
 (1, 15): 0,
 (1, 16): 0.05,
 (1, 17): 0,
 (1, 18): 0,
 (1, 19): 3,
 (1, 20): 3,
 (2, 3): 0.1,
 (2, 4): 3,
 (2, 5): 0.1,
 (2, 6): 0.1,
 (2, 7): 3,
 (2, 8): 0,
 (2, 9): 3,
 (2, 10): 0.1,
 (2, 11): 0,
 (2, 12): 3,
 (2, 13): 0,
 (2, 14): 0,
 (2, 15): 3,
 (2, 16): 0,
 (2, 17): 0.1,
 (2, 18): 3,
 (2, 19): 0,
 (2, 20): 0,
 (3, 4): 0,
 (3, 5): 0,
 (3, 6): 0,
 (3, 7): 0,
 (3, 8): 2,
 (3, 9): 0,
 (3, 10): 0,
 (3, 11): 2,
 (3, 12): 0,
 (3, 13): 2,
 (3, 14): 0.1,
 (3, 15): 0,
 (3, 16): 2,
 (3, 17): 0,
 (3, 18): 0,
 (3, 19): 0.1,
 (3, 20): 0.1,
 (4, 5): 0,
 (4, 6): 0,
 (4, 7): 0,
 (4, 8): 0.05,
 (4, 9): 0,
 (4, 10): 0,
 (4, 11): 0.05,
 (4, 12): 0,
 (4, 13): 0.05,
 (4, 14): 3,
 (4, 15): 0,
 (4, 16): 0.05,
 (4, 17): 0,
 (4, 18): 0,
 (4, 19): 3,
 (4, 20): 3,
 (5, 6): 0,
 (5, 7): 0,
 (5, 8): 2,
 (5, 9): 0,
 (5, 10): 0,
 (5, 11): 2,
 (

## Non-Crossing Constraint

In [15]:
m.addConstrs(quicksum(x[k,l] for k in range(i+1,j) for l in [i for i in list(range(1,len(s)+1)) if i >= j]) <= (10**6)*(1-x[i,j]) for i in range(1,len(s)+1) for j in range(i+1,len(s)+1))

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model

## Character in At most one pair

In [16]:
m.addConstrs(quicksum([x[i,k] for i in range(1,k)]) + quicksum([x[k,j] for j in list(range(k+1,len(s)+1))]) <= 1 for k in ONE_N)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>}

# Objective

In [17]:
m.setObjective((quicksum(f[i,j]*x[i,j] for i in ONE_N for j in list(range(i+1,len(s)+1)))),GRB.MAXIMIZE)

In [18]:
# Optimize the model
m.optimize()
# Print the result
status_code = {1: 'LOADED', 2: 'OPTIMAL', 3: 'INFEASIBLE', 4: 'INF_OR_UNBD', 5: 'UNBOUNDED'}
status = m.status
print("The optimization status is {}".format(status_code[status]))
if status == 2:
# Retrieve variables value
    print("Optimal solution:")
    for v in m.getVars():
        if (v.x == 1):
            print("%s = %g" % (v.varName, v.x))
    print("Optimal objective value:\n{}".format(m.objVal))
    print("Runtime is:", m.Runtime)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 210 rows, 190 columns and 5940 nonzeros
Model fingerprint: 0x275ab595
Variable types: 0 continuous, 190 integer (190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [5e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 0.3000000
Presolve removed 135 rows and 118 columns
Presolve time: 0.02s
Presolved: 75 rows, 72 columns, 1225 nonzeros
Variable types: 0 continuous, 72 integer (72 binary)
Found heuristic solution: objective 11.0000000

Root relaxation: objective 1.884928e+01, 61 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   18.84928    0   23   11.00000   18.84928  71.4%

In [19]:
m.write("check.lp")